In [338]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_regression


In [ ]:
df = pd.read_csv('../../data/processed/combined_data_v3_nearby_central_city.csv')

In [340]:
df.head()

,id,price,bedrooms,bathrooms,land_area,address_subdivision,address_locality,address_line_2,category_name,is_parent,...,Sisowath_Riverside_Park_nearest,Sisowath_Riverside_Park_1_2km,Sisowath_Riverside_Park_2_3km,Sisowath_Riverside_Park_3_5km,Sisowath_Riverside_Park_5_10km,Phnom_Penh_Airport_nearest,Phnom_Penh_Airport_1_2km,Phnom_Penh_Airport_2_3km,Phnom_Penh_Airport_3_5km,Phnom_Penh_Airport_5_10km
0,185714.0,1100000.0,7.0,7.0,124.0,Phnom Penh,Daun Penh,Chakto Mukh,Shophouse,False,...,0,1,0,0,0,0,0,0,0,1
1,185539.0,680000.0,4.0,5.0,80.0,Phnom Penh,Chamkarmon,BKK 3,House,False,...,0,0,1,0,0,0,0,0,0,0
2,217752.0,550000.0,3.0,4.0,66.0,Phnom Penh,Daun Penh,Chey Chumneah,Retreat,False,...,0,1,0,0,0,0,0,0,0,1
3,228897.0,750000.0,9.0,10.0,116.0,Phnom Penh,Chamkarmon,Tonle Bassac,Villa,False,...,0,0,0,1,0,0,0,0,0,1
4,190024.0,420000.0,5.0,6.0,65.0,Phnom Penh,Chroy Changvar,Chroy Changvar,House,False,...,0,1,0,0,0,0,0,0,0,0


In [341]:
df = df[df['price_per_m2'] >= 100].copy()

In [342]:
df.drop_duplicates(inplace=True)

In [343]:
# df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].fillna(0)

In [344]:
# Calculate mean, max, min of 'price_per_m2' grouped by 'h_id'
price_stats = df.groupby('address_line_2')['price_per_m2'].agg(['mean','median', 'max', 'min']).reset_index()

# Merge these stats back to the original dataframe on 'h_id'
df = df.merge(price_stats, on='address_line_2', how='left', suffixes=('', '_price_per_m2'))

# The columns 'mean', 'max', 'min' are now added to df

In [345]:
df.drop(columns=['id', 'price', 'bedrooms', 'bathrooms','address_subdivision', 'price_display', 'information', 'price_display', 'geometry', 'index_right'], inplace=True)

In [346]:
df

,land_area,address_locality,address_line_2,category_name,is_parent,type,latitude,longitude,price_per_m2,population,...,Sisowath_Riverside_Park_5_10km,Phnom_Penh_Airport_nearest,Phnom_Penh_Airport_1_2km,Phnom_Penh_Airport_2_3km,Phnom_Penh_Airport_3_5km,Phnom_Penh_Airport_5_10km,mean,median,max,min
0,124.0,Daun Penh,Chakto Mukh,Shophouse,False,residential,11.575610,104.920250,8870.967742,16252.0,...,0,0,0,0,0,1,3908.371821,3829.257246,8870.967742,104.166667
1,80.0,Chamkarmon,BKK 3,House,False,residential,11.550000,104.930000,8500.000000,7658.0,...,0,0,0,0,0,0,2730.697070,2129.508909,8500.000000,370.370370
2,66.0,Daun Penh,Chey Chumneah,Retreat,False,residential,11.575610,104.920250,8333.333333,16252.0,...,0,0,0,0,0,1,3818.434615,2963.873450,8333.333333,1012.658228
3,116.0,Chamkarmon,Tonle Bassac,Villa,False,residential,11.544500,104.913586,6465.517241,23239.0,...,0,0,0,0,0,1,2945.379851,2878.787879,6465.517241,254.777070
4,65.0,Chroy Changvar,Chroy Changvar,House,False,residential,11.580000,104.930000,6461.538462,5351.0,...,0,0,0,0,0,0,2047.471213,1754.120879,6461.538462,170.135882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,833.0,Toul Kork,Boeung Kak 1,Condo,False,residential,11.550000,104.930000,108.031212,7658.0,...,0,0,0,0,0,0,2376.653525,2000.000000,6000.000000,108.031212
3338,800.0,Chamkarmon,BKK 1,project,True,residential,11.545695,104.923107,106.250000,13796.0,...,0,0,0,0,0,1,3009.237377,2815.533981,6428.571429,100.000000
3339,480.0,Daun Penh,Chakto Mukh,Business,False,commercial,11.575837,104.920096,104.166667,16252.0,...,0,0,0,0,0,1,3908.371821,3829.257246,8870.967742,104.166667
3340,7960.0,Dangkao,Tien,Land/Development,False,residential,11.439340,104.849064,100.000000,751.0,...,0,0,0,0,0,0,180.000000,120.000000,320.000000,100.000000


In [347]:
# df.drop(columns=['price', 'address_line_2', 'id', 'address_subdivision', 'address_locality', 'information', 'geometry', 'index_right', 'price_display', 'h_id'], inplace=True)

In [348]:
df.dropna(inplace=True)

In [349]:
y = df[['price_per_m2']]
X = df.drop(columns=['price_per_m2'])

In [350]:
X.describe()

,land_area,latitude,longitude,population,nearest_cafe,n_cafe_in_1km,n_cafe_in_1km_to_2km,n_cafe_in_2km_to_3km,n_cafe_in_3km_to_5km,nearest_gas_station,...,Sisowath_Riverside_Park_5_10km,Phnom_Penh_Airport_nearest,Phnom_Penh_Airport_1_2km,Phnom_Penh_Airport_2_3km,Phnom_Penh_Airport_3_5km,Phnom_Penh_Airport_5_10km,mean,median,max,min
count,3325.000000,3325.000000,3325.000000,3325.000000,3325.00000,3325.000000,3325.000000,3325.000000,3325.000000,3325.000000,...,3325.000000,3325.000000,3325.000000,3325.000000,3325.000000,3325.000000,3325.000000,3325.000000,3325.000000,3325.000000
mean,589.848202,11.554426,104.912497,12356.590376,3.25594,6.890827,20.675188,29.349173,66.389774,1.369925,...,0.280602,0.001805,0.007820,0.006316,0.042105,0.541353,2240.769270,2022.064255,5527.894712,411.885884
std,3353.124461,0.035503,0.029208,8770.041576,3.32845,5.970479,13.707943,17.437024,30.012346,1.749419,...,0.449361,0.042448,0.088095,0.079232,0.200860,0.498362,645.293262,663.971177,1419.382538,342.676441
min,1.000000,11.430122,104.748841,3.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,164.932003,120.000000,179.864005,100.000000
25%,74.200000,11.534079,104.898970,4621.000000,0.00000,1.000000,9.000000,9.000000,50.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1858.919985,1661.111111,4999.984375,195.396707
50%,115.500000,11.550000,104.916255,8396.000000,2.00000,5.000000,21.000000,35.000000,71.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2145.079724,1888.372093,6222.222222,278.448116
75%,288.000000,11.575610,104.930000,20064.000000,6.00000,12.000000,32.000000,44.000000,85.000000,2.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2730.697070,2470.396904,6428.571429,500.000000
max,104635.000000,11.720574,104.996857,39519.000000,14.00000,26.000000,59.000000,67.000000,125.000000,11.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3949.618811,3925.511588,8870.967742,2666.666667


In [351]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train.shape, X_test.shape

In [352]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns


In [353]:
cat_cols

Index(['address_locality', 'address_line_2', 'category_name', 'type', 'h_id'], dtype='object')

In [354]:
# Temporarily encode categorical columns for feature selection
X_temp = X.copy()
if len(cat_cols) > 0:
    encoder = OrdinalEncoder()
    X_temp[cat_cols] = encoder.fit_transform(X_temp[cat_cols])


In [355]:
df.isnull().sum()   

land_area                    0
address_locality             0
address_line_2               0
category_name                0
is_parent                    0
                            ..
Phnom_Penh_Airport_5_10km    0
mean                         0
median                       0
max                          0
min                          0
Length: 195, dtype: int64

In [356]:
import plotly.express as px

cor = X_temp.corr()

fig = px.imshow(
    cor,
    color_continuous_scale='RdBu_r',
    aspect='auto',
    title='Correlation Matrix',
    width=1200,
    height=1200
)
fig.write_html("correlation_matrix.html")
fig.show()

In [357]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    cor_matrix = dataset.corr()
    for i in range(len(cor_matrix.columns)):
        for j in range(i):
            if abs(cor_matrix.iloc[i, j]) > threshold:  # We are interested in absolute correlation
                colname = cor_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [358]:
df.head()

,land_area,address_locality,address_line_2,category_name,is_parent,type,latitude,longitude,price_per_m2,population,...,Sisowath_Riverside_Park_5_10km,Phnom_Penh_Airport_nearest,Phnom_Penh_Airport_1_2km,Phnom_Penh_Airport_2_3km,Phnom_Penh_Airport_3_5km,Phnom_Penh_Airport_5_10km,mean,median,max,min
0,124.0,Daun Penh,Chakto Mukh,Shophouse,False,residential,11.57561,104.920250,8870.967742,16252.0,...,0,0,0,0,0,1,3908.371821,3829.257246,8870.967742,104.166667
1,80.0,Chamkarmon,BKK 3,House,False,residential,11.55000,104.930000,8500.000000,7658.0,...,0,0,0,0,0,0,2730.697070,2129.508909,8500.000000,370.370370
2,66.0,Daun Penh,Chey Chumneah,Retreat,False,residential,11.57561,104.920250,8333.333333,16252.0,...,0,0,0,0,0,1,3818.434615,2963.873450,8333.333333,1012.658228
3,116.0,Chamkarmon,Tonle Bassac,Villa,False,residential,11.54450,104.913586,6465.517241,23239.0,...,0,0,0,0,0,1,2945.379851,2878.787879,6465.517241,254.777070
4,65.0,Chroy Changvar,Chroy Changvar,House,False,residential,11.58000,104.930000,6461.538462,5351.0,...,0,0,0,0,0,0,2047.471213,1754.120879,6461.538462,170.135882


In [359]:
cor_features = correlation(X_temp, 0.7)

In [360]:
# # Feature selection 
# selector = SelectKBest(mutual_info_regression, k=30)
# selector.fit(X_temp, y)
# selected_features = X.columns[selector.get_support()]


In [361]:
X.drop(columns=cor_features, inplace=True)

In [362]:
X

,land_area,address_locality,address_line_2,category_name,is_parent,type,latitude,longitude,population,h_id,...,Vattanac_Tower_2_3km,Vattanac_Tower_3_5km,Royal_Palace_2_3km,Phnom_Penh_Airport_nearest,Phnom_Penh_Airport_1_2km,Phnom_Penh_Airport_2_3km,Phnom_Penh_Airport_3_5km,mean,max,min
0,124.0,Daun Penh,Chakto Mukh,Shophouse,False,residential,11.575610,104.920250,16252.0,8865846aadfffff,...,0,0,0,0,0,0,0,3908.371821,8870.967742,104.166667
1,80.0,Chamkarmon,BKK 3,House,False,residential,11.550000,104.930000,7658.0,8865846ae9fffff,...,1,0,0,0,0,0,0,2730.697070,8500.000000,370.370370
2,66.0,Daun Penh,Chey Chumneah,Retreat,False,residential,11.575610,104.920250,16252.0,8865846aadfffff,...,0,0,0,0,0,0,0,3818.434615,8333.333333,1012.658228
3,116.0,Chamkarmon,Tonle Bassac,Villa,False,residential,11.544500,104.913586,23239.0,8865846ac7fffff,...,0,1,1,0,0,0,0,2945.379851,6465.517241,254.777070
4,65.0,Chroy Changvar,Chroy Changvar,House,False,residential,11.580000,104.930000,5351.0,886584685bfffff,...,0,0,0,0,0,0,0,2047.471213,6461.538462,170.135882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,833.0,Toul Kork,Boeung Kak 1,Condo,False,residential,11.550000,104.930000,7658.0,8865846ae9fffff,...,1,0,0,0,0,0,0,2376.653525,6000.000000,108.031212
3338,800.0,Chamkarmon,BKK 1,project,True,residential,11.545695,104.923107,13796.0,8865846ac5fffff,...,0,1,1,0,0,0,0,3009.237377,6428.571429,100.000000
3339,480.0,Daun Penh,Chakto Mukh,Business,False,commercial,11.575837,104.920096,16252.0,8865846aadfffff,...,0,0,0,0,0,0,0,3908.371821,8870.967742,104.166667
3340,7960.0,Dangkao,Tien,Land/Development,False,residential,11.439340,104.849064,751.0,88658460c5fffff,...,0,0,0,0,0,0,0,180.000000,320.000000,100.000000


In [363]:
# One-hot encode only the selected categorical features
selected_cat_cols = [col for col in X if col in cat_cols]
X_final = pd.get_dummies(X, columns=cat_cols, drop_first=True)
X_final = X_final.astype(int)


In [364]:
X_final

,land_area,is_parent,latitude,longitude,population,nearest_cafe,n_cafe_in_1km,n_cafe_in_3km_to_5km,nearest_gas_station,n_gas_station_in_1km,...,h_id_8865847887fffff,h_id_88658478a3fffff,h_id_88658478b3fffff,h_id_88658478b7fffff,h_id_88658478bbfffff,h_id_8865847993fffff,h_id_886586a691fffff,h_id_886586a693fffff,h_id_886586a699fffff,h_id_886586a69bfffff
0,124,0,11,104,16252,11,10,76,0,14,...,0,0,0,0,0,0,0,0,0,0
1,80,0,11,104,7658,7,14,62,2,1,...,0,0,0,0,0,0,0,0,0,0
2,66,0,11,104,16252,11,10,76,0,14,...,0,0,0,0,0,0,0,0,0,0
3,116,0,11,104,23239,7,12,85,1,13,...,0,0,0,0,0,0,0,0,0,0
4,65,0,11,104,5351,0,6,87,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,833,0,11,104,7658,7,14,62,2,1,...,0,0,0,0,0,0,0,0,0,0
3338,800,1,11,104,13796,3,15,81,3,4,...,0,0,0,0,0,0,0,0,0,0
3339,480,0,11,104,16252,11,11,77,0,14,...,0,0,0,0,0,0,0,0,0,0
3340,7960,0,11,104,751,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [365]:
df_pearson_correlation = pd.concat([X_final, y], axis=1)

In [366]:
df_pearson_correlation.dropna(inplace=True)

In [367]:
df_pearson_correlation

,land_area,is_parent,latitude,longitude,population,nearest_cafe,n_cafe_in_1km,n_cafe_in_3km_to_5km,nearest_gas_station,n_gas_station_in_1km,...,h_id_88658478a3fffff,h_id_88658478b3fffff,h_id_88658478b7fffff,h_id_88658478bbfffff,h_id_8865847993fffff,h_id_886586a691fffff,h_id_886586a693fffff,h_id_886586a699fffff,h_id_886586a69bfffff,price_per_m2
0,124,0,11,104,16252,11,10,76,0,14,...,0,0,0,0,0,0,0,0,0,8870.967742
1,80,0,11,104,7658,7,14,62,2,1,...,0,0,0,0,0,0,0,0,0,8500.000000
2,66,0,11,104,16252,11,10,76,0,14,...,0,0,0,0,0,0,0,0,0,8333.333333
3,116,0,11,104,23239,7,12,85,1,13,...,0,0,0,0,0,0,0,0,0,6465.517241
4,65,0,11,104,5351,0,6,87,0,0,...,0,0,0,0,0,0,0,0,0,6461.538462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,833,0,11,104,7658,7,14,62,2,1,...,0,0,0,0,0,0,0,0,0,108.031212
3338,800,1,11,104,13796,3,15,81,3,4,...,0,0,0,0,0,0,0,0,0,106.250000
3339,480,0,11,104,16252,11,11,77,0,14,...,0,0,0,0,0,0,0,0,0,104.166667
3340,7960,0,11,104,751,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,100.000000


In [ ]:
# df_pearson_correlation.to_csv('../../data/preprocessed/realestates_kh_Pearson_v2.csv', index=False)